<a href="https://colab.research.google.com/github/AU2302DM-TeamKYJ/TeamProject-SteamGameRecommander/blob/main/Lab/231115_TestRun_bySHY.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import urllib.request
import json
from pandas import json_normalize
from urllib.parse import urlencode, quote_plus, unquote

In [ ]:
api = 'http://api.steampowered.com/ISteamUser/GetFriendList/v0001/'
steamid = '76561198003081163' # 스트리머 우왁굳
queryParams = '?' + urlencode({quote_plus('key') : '',
                               quote_plus('steamid') : steamid,
                               quote_plus('relationship') : 'friend'
                               })
url = api+queryParams

In [ ]:
print(url)

In [ ]:
already_scanned =

In [ ]:
#데이터를 요청하고 변환합니다.
text = urllib.request.urlopen(url).read().decode('utf-8')
json_return = json.loads(text)
# get_data = json_return.get('friendlist')
# df = json_normalize(get_data['body']['items'])

In [ ]:
friends_list = []

friends_data = json_return.get('friendslist').get('friends')
for friend in friends_data:
    friends_list.append(friend.get('steamid'))
friends_list = np.unique(friends_list)

In [ ]:
print(friends_list)

In [ ]:
api = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
steamid = '76561197973315893' # 76561197973315893 : 스트리머 우왁굳의 친구 리스트 중 한 명
queryParams = '?' + urlencode({quote_plus('key') : '',
                               quote_plus('steamid') : steamid,
                               quote_plus('include_played_free_games') : 1,
                               quote_plus('format') : 'json'
                               })
url = api+queryParams

In [ ]:
print(json_return)

In [ ]:
print(json_return.get('response') == {})

True


In [ ]:
friends_csv = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/DM2302/Project/steamIds.csv')

gamers = []

for user in friends_csv['Steam ID']:
    api = 'http://api.steampowered.com/IPlayerService/GetOwnedGames/v0001/'
    steamid = user # 76561197973315893 : 스트리머 우왁굳의 친구 리스트 중 한 명
    queryParams = '?' + urlencode({quote_plus('key') : '',
                                  quote_plus('steamid') : steamid,
                                  quote_plus('include_played_free_games') : 1,
                                  quote_plus('format') : 'json'
                                  })
    url = api+queryParams

    text = urllib.request.urlopen(url).read().decode('utf-8')
    json_return = json.loads(text)

    if json_return.get('response') == {}:
        continue

    played_game_list = []

    owned_game_list = json_return.get('response').get('games')
    for game in owned_game_list:
        if game.get('playtime_forever') == 0:
            continue
        played_game_list.append(game.get('appid'))

    gamers.append({steamid : played_game_list})

In [ ]:
print(gamers)

In [ ]:
import csv

f = open('out2.csv', 'w', newline='')

writer = csv.writer(f)
writer.writerows(gamers)
f.close()

In [75]:
df.to_csv('/content/drive/MyDrive/Colab Notebooks/DM2302/Project/id_premitive.csv')

In [66]:
pre_df = []

for user in gamers:
    for id in list(user.keys()):
        for game in user[id]:
            pre_df.append([id, game])

In [74]:
df = pd.DataFrame(pre_df)
df.columns = ['UserID', 'GameID']
df.head()

,UserID,GameID
0,76561197967479980,10
1,76561197967479980,220
2,76561197967479980,240
3,76561197967479980,4000
4,76561197967479980,400


In [ ]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

import pyspark
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext, SparkConf

In [3]:
# create the session
conf = SparkConf().set("spark.ui.port", "4050")

# create the context
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [79]:
py_df = spark.createDataFrame(df)

In [4]:
py_df = spark.read.csv('/content/drive/MyDrive/Colab Notebooks/DM2302/Project/id_premitive.csv', header=True, inferSchema=True)
py_df = py_df.drop('_c0')

In [5]:
py_df.printSchema()
py_df.show()

root
 |-- UserID: long (nullable = true)
 |-- GameID: integer (nullable = true)

+-----------------+------+
|           UserID|GameID|
+-----------------+------+
|76561197967479980|    10|
|76561197967479980|   220|
|76561197967479980|   240|
|76561197967479980|  4000|
|76561197967479980|   400|
|76561197967479980|   420|
|76561197967479980| 40800|
|76561197967479980|   620|
|76561197967479980|105600|
|76561197967479980|   730|
|76561197967479980|204300|
|76561197967479980| 51100|
|76561197967479980|263280|
|76561197967479980|211820|
|76561197967479980|286160|
|76561197967479980|304050|
|76561197967479980|311690|
|76561197967479980|271590|
|76561197967479980|234630|
|76561197967479980|378860|
+-----------------+------+
only showing top 20 rows



In [6]:
data_grouped = py_df.groupBy("UserID").agg(collect_set('GameID').alias('GamesID'))

In [7]:
data_grouped.printSchema()
data_grouped.show()

root
 |-- UserID: long (nullable = true)
 |-- GamesID: array (nullable = false)
 |    |-- element: integer (containsNull = false)

+-----------------+--------------------+
|           UserID|             GamesID|
+-----------------+--------------------+
|76561197960325732|[203160, 10, 4571...|
|76561197964253148|[1591520, 1104380...|
|76561197966333607|[1104380, 674750,...|
|76561197967055907|[266510, 377360, ...|
|76561197967479980|[286160, 698610, ...|
|76561197968304848|[1938090, 335430,...|
|76561197969345471|[214490, 57400, 1...|
|76561197969699255|[941570, 110800, ...|
|76561197971565071|[396480, 34330, 1...|
|76561197973315893|[34330, 289130, 5...|
|76561197975276751|[244210, 1938090,...|
|76561197977298897|[292120, 292730, ...|
|76561197985535105|[248820, 291410, ...|
|76561197987255937|[214490, 498240, ...|
|76561197987498968|[291410, 262410, ...|
|76561197988122922|[298110, 1046930,...|
|76561197989992100|[224580, 8930, 19...|
|76561197990912803|[360170, 304390, ...|
|7656119

In [14]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="GamesID", minSupport=0.5, minConfidence=0.9)
model = fpGrowth.fit(data_grouped)

In [15]:
freq_01 = model.freqItemsets
asso_01 = model.associationRules
print(f"Count of frequent itemsets: {freq_01.count()}\nCount of association rules: {asso_01.count()}\n")
freq_01.sort(desc('freq')).show()
asso_01.sort(desc('lift')).show()

Count of frequent itemsets: 22
Count of association rules: 9

+------------------+----+
|             items|freq|
+------------------+----+
|          [578080]| 290|
|             [730]| 229|
|             [550]| 225|
|     [730, 578080]| 217|
|          [431960]| 212|
|     [550, 578080]| 211|
|             [440]| 208|
|          [271590]| 203|
|  [431960, 578080]| 200|
|     [440, 578080]| 195|
|  [271590, 578080]| 188|
|        [550, 730]| 179|
|          [359550]| 177|
|          [218620]| 173|
|  [359550, 578080]| 172|
|        [440, 550]| 171|
|        [440, 730]| 171|
|[550, 730, 578080]| 170|
|             [570]| 165|
|            [4000]| 165|
+------------------+----+
only showing top 20 rows

+----------+----------+------------------+------------------+------------------+
|antecedent|consequent|        confidence|              lift|           support|
+----------+----------+------------------+------------------+------------------+
|  [359550]|  [578080]|0.9717514124293786|1.0

In [ ]:
# 578080 : 배틀그라운드        - 무료게임 O
# 359550 : 레인보우식스 시즈    - 무료게임 X
# 440 : 팀 포트리스 2         - 무료게임 O
# 550 : 레프트 포 데드 2       - 무료게임 O...?
# 730 : 카운터 스트라이크 2     - 무료게임 O
# 431960 : 월페이퍼 엔진       - 무료게임 X...?
# 218620 : 페이데이 2         - 무료게임 X
# 271590 : GTA 5            - 무료게임 X
